In [9]:
#Import the packages I will use
import pandas as pd
import folium
from folium import IFrame
from pykml import parser
import csv
import webbrowser

In [10]:
#Browse data to look for certain key information to keep
filename = 'alt_fuel_stations (Jul 29 2021).csv'
df = pd.read_csv(filename, low_memory=False)
df


,Fuel Type Cotion Directions,City,State,ZIP,Plus4,Station Phone,Status Code,Expected Date,Groups With Access Code,Access Days Time,...,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Accessde,Station Name,Street Address,Intersec
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,MD,NaN,NaN,NaN
1,CNG,PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Ave, left on...",Atlanta,GA,30303,NaN,770-350-3000,E,...,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN,NaN
2,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324,NaN,NaN,E,...,NaN,NaN,NaN,NaN,Q,3000,LD,NaN,NaN,NaN
3,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,HD,NaN,NaN,NaN
4,CNG,Clean Energy - Texas Department of Transportation,7721A Washington St,"I-10, Washington Ave exit, 1.5 blocks to the s...",Houston,TX,77007,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,Q,3000 3600,MD,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56795,E85,Sheetz Store #713,10515 Sharpsburg Pk,NaN,Hagerstown,MD,21740,NaN,240-513-7047,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56796,E85,Kwik Trip #1062,1580 E Washington St,NaN,West Bend,WI,53095,NaN,262-334-0999,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56797,E85,76,675 N Lake Ave,NaN,Pasadena,CA,91101,NaN,626-578-1519,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56798,E85,Lafferty Race Fuels,8800 S Osage St,NaN,Amarillo,TX,79118,NaN,806-674-7536,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#Cleaning the Data
#Identify the columns to keep from original df
columns_to_keep = ['City', 'ID', 'Updated At','Fuel Type Cotion Directions', 'State']  

# Select only the specified columns
stations_df= df[columns_to_keep]
    
# Print the DataFrame with selected columns
stations_df.sample(15)
type(stations_df['City'])
type(stations_df['Updated At'])

# I need to remake new columns to make the DF easier to understand
stations_df['lat'] = stations_df['ID']
stations_df['lon'] = stations_df['Updated At']
stations_df['Address'] = stations_df['State']
stations_df['Station_Name'] = stations_df['City']
stations_df['Fuel Type'] = stations_df['Fuel Type Cotion Directions']
stations_df.head(10)

/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_21107/2889185085.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_df['lat'] = stations_df['ID']
/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_21107/2889185085.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_df['lon'] = stations_df['Updated At']
/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_21107/2889185085.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,City,ID,Updated At,Fuel Type Cotion Directions,State,lat,lon,Address,Station_Name,Fuel Type
0,Spire - Montgomery Operations Center,32.367916,-86.267021,CNG,2951 Chestnut St,32.367916,-86.267021,2951 Chestnut St,Spire - Montgomery Operations Center,CNG
1,PS Energy - Atlanta,33.745843,-84.398837,CNG,340 Whitehall St,33.745843,-84.398837,340 Whitehall St,PS Energy - Atlanta,CNG
2,Metropolitan Atlanta Rapid Transit Authority,33.821911,-84.367461,CNG,2424 Piedmont Rd NE,33.821911,-84.367461,2424 Piedmont Rd NE,Metropolitan Atlanta Rapid Transit Authority,CNG
3,United Parcel Service,33.760256,-84.543822,CNG,270 Marvin Miller Dr,33.760256,-84.543822,270 Marvin Miller Dr,United Parcel Service,CNG
4,Clean Energy - Texas Department of Transportation,29.779918,-95.435385,CNG,7721A Washington St,29.779918,-95.435385,7721A Washington St,Clean Energy - Texas Department of Transportation,CNG
5,Arkansas Oklahoma Gas Corp,35.362213,-94.375338,CNG,2100 S Waldron Rd,35.362213,-94.375338,2100 S Waldron Rd,Arkansas Oklahoma Gas Corp,CNG
6,Clean Energy - Logan International Airport,42.374706,-71.026549,CNG,1000 Cottage St Ext,42.374706,-71.026549,1000 Cottage St Ext,Clean Energy - Logan International Airport,CNG
7,Clean Energy - Everett - National Grid,42.393167,-71.064352,CNG,16 Rover St,42.393167,-71.064352,16 Rover St,Clean Energy - Everett - National Grid,CNG
8,Clean Energy - Greenpoint - National Grid,40.718037,-73.932309,CNG,287 Maspeth Ave,40.718037,-73.932309,287 Maspeth Ave,Clean Energy - Greenpoint - National Grid,CNG
9,Canarsie - National Grid,40.645540,-73.918344,CNG,8424 Ditmas Ave,40.645540,-73.918344,8424 Ditmas Ave,Canarsie - National Grid,CNG


In [12]:
#renamed the columns to "lat" and "lon", dropped old columns

columns_to_drop = ['ID', 'Updated At','City', 'State', 'Fuel Type Cotion Directions']
stations_df.drop(columns=columns_to_drop, inplace=True)
ev_df = pd.DataFrame(stations_df)

/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_21107/3868340838.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_df.drop(columns=columns_to_drop, inplace=True)


In [13]:
#changed lat and lon to floats
ev_df['lat'] = ev_df['lat'].astype(float)
ev_df['lon'] = ev_df['lon'].astype(float)

#save the data as a new CSV
ev_df.to_csv('New_EV_Data.csv', index=False)


In [14]:
#creating a Test DataFrame to analyze the cleaned data
test_df =  ev_df.sample(1000)


In [15]:
#Create the map to host data on.

m = folium.Map(location=[test_df['lat'].mean(), test_df['lon'].mean()], zoom_start=4)

# Add markers for each location in the DataFrame
#USing the test_df for visualization purpose.
for index, row in test_df.iterrows():
    popup_text = f"<b>{row['Station_Name']}</b><br>Fuel Type: {row['Fuel Type']}<br>Address: {row['Address']}"
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=popup_text,
        tooltip=row['Station_Name']
    ).add_to(m)

# Save and open the map in a new window
# Save the map to an HTML file
m.save('map.html')

#Saves a local File of a Interactive map to analyze locations of
#Fuel Service Stations
webbrowser.open('map.html')

#Displays local map
m

